In [4]:
import requests,csv,json,re
from bs4 import BeautifulSoup
s = requests.Session()

In [5]:
def get_xfToken(url):
    kk = s.get(url)
    soup = BeautifulSoup(kk.text,'lxml')
    _xfToken = soup.select_one('[name="_xfToken"]')['value']
    return _xfToken

def login(admin_id,admin_pass):
    xfToken = get_xfToken('http://joonbud.com/login/login')  
    loginUrl = 'http://joonbud.com/login/login'
    payload = {
        '_xfToken':xfToken,
        'login':admin_id,
        'password':admin_pass,
        'remember':1,
        '_xfRedirect':'http://joonbud.com/'
    }
    res = s.post(loginUrl,data=payload,verify=False)
    
    if res.status_code == 200 or res.status_code == 303:
        cookies = res.history[0].headers.get('Set-Cookie')
        print(f'{admin_id}@{admin_pass} login --> statusCode {res.status_code}')
        xf_user = re.findall(r'xf_user.+?(?=;)',cookies)[0].removeprefix('xf_user=')
        xf_session = re.findall(r'xf_session.+?(?=;)',cookies)[0].removeprefix('xf_session=')
        # s.headers.update({'Cookie':cookies})
        cookies = {"xf_user":xf_user,"xf_session":xf_session}
        s.cookies.update(cookies)

In [6]:
adminID = 'mustafadev'
adminPSW = 'mustafadevupwork'
login(adminID,adminPSW)

mustafadev@mustafadevupwork login --> statusCode 200


In [7]:
def addProfessor(row):
    title,tag_line = row[0],row[1]
    xfToken = get_xfToken('http://joonbud.com/professors/categories/saddleback-college.2/add') 
    addPayload = {
        '_xfToken':xfToken,
        'prefix_id':1,
        'title':title+'A',
        'tag_line':tag_line,
        'resource_type':'fileless',
        'description':'<p>-</p>',
        # 'attachment_hash':'a857f76edb39b845ce4439ed61916cde',
        # 'attachment_hash_combined':'{"type":"resource_update","context":{"resource_category_id":2},"hash":"a857f76edb39b845ce4439ed61916cde"}',
        '_xfRequestUri':'/professors/categories/saddleback-college.2/add',
        
    }

    addresp = s.post('http://joonbud.com/professors/categories/saddleback-college.2/add',data=addPayload)
    soup = BeautifulSoup(addresp.content,'lxml')
    title = soup.title.text
    print(title)

In [5]:
# with open('kk.html','w') as f : f.write(addresp.text)

In [8]:
# import mechanize
# br = mechanize.Browser()
# br.open('http://www.imdb.com/title/tt0108778/')


In [8]:
def readTrackLog():
    with open('TrackLogg.csv') as trackLoggFile:
        trackRows = list(csv.reader(trackLoggFile))
        return trackRows
    
    
def updateTrackLogger(row):
    with open('TrackLogg.csv', 'a',encoding='utf-8',newline='') as file:
        csv.writer(file).writerow(row)

In [11]:
trackRows = readTrackLog()
with open('addingInput.csv') as inputDataFile:
    inputRows = list(csv.reader(inputDataFile))
    for row in inputRows:
        if row not in trackRows:
            print(f'Row::{row}')
            addProfessor(row)
            updateTrackLogger(row)
            trackRows = readTrackLog()
            print(f'Added::{row}')
        else:
            print(f'skipped::{row}')

Row::['Teacher Name', 'Department + Class']
Oops! We ran into some problems. | Testserver
Added::['Teacher Name', 'Department + Class']
skipped::['Ruth Wrightsman BIO15', 'Science BIO15']
skipped::['Ruth Wrightsman MICROBIO', 'Science MICROBIO']
skipped::['Ruth Wrightsman BIO15', 'Science BIO15']
skipped::['Arnold McMahon HUM1', 'Philosophy HUM1']
skipped::['Arnold McMahon HUMMANITIES', 'Philosophy HUMMANITIES']
skipped::['Arnold McMahon PHIL101', 'Philosophy PHIL101']
skipped::['Arnold McMahon HUM25', 'Philosophy HUM25']
skipped::['Arnold McMahon PHIL1035', 'Philosophy PHIL1035']
skipped::['Arnold McMahon HUM22', 'Philosophy HUM22']
skipped::['Arnold McMahon PHIL22', 'Philosophy PHIL22']
skipped::['Arnold McMahon LOGIC1', 'Philosophy LOGIC1']
skipped::['Arnold McMahon ETHICS', 'Philosophy ETHICS']
skipped::['Arnold McMahon PHIL15', 'Philosophy PHIL15']
skipped::['Arnold McMahon ETHICS', 'Philosophy ETHICS']
skipped::['Arnold McMahon HUM22', 'Philosophy HUM22']
skipped::['Arnold McMaho

KeyboardInterrupt: 